In [1]:
import json
import os 
import requests
import time
from datetime import datetime,timedelta
from datetime import date
import re
import sys
import urllib, urllib.request, urllib.parse
import random
from scrawl import *
    
# Date and time
start_time = time.time()
current_time = datetime.now().strftime("%H-%M-%S")
created_on = date.today().strftime("%Y-%m-%d")

# client_id = sys.argv[1]
client_id = '5f69d22ef472d6646f577fa6'  # Europe
site = 'en_ndrc_gov_cn_publication'
site_name = 'National Development and Reform Commission Academy of Macroeconomic Research (NDRC) (China)'
c = Crawl()  # creating object

# create directories to store logs.
log_path = c.create_directories(project_path, client_id, site)

# create image directories
image_directory = c.create_image_directories(project_path)

# logger
logger = log_func(log_path, created_on, current_time)
logger.info("Process Started ...\n")

# initialize variables
skipped_due_to_headline = 0
skipped_due_to_content = 0
skipped_due_to_date = 0
missing_overall_tonality = 0
no_of_data = 0
duplicate_data = 0  
unable_to_fetch_article_url = 0
unable_to_fetch_cat_url  = 0
publish_source = 'en.ndrc.gov.cn'
country = 'China'
language = 'English'
images_path = []
foot_fall = c.get_foot_fall(publish_source)

# archive page
rss_page = c.download_page('https://en.ndrc.gov.cn/newsrelease_8232/index.html')
urls =['https://en.ndrc.gov.cn/news/pressreleases/',
      'https://en.ndrc.gov.cn/news/activities/',
      'https://en.ndrc.gov.cn/news/mediarusources/']
for url in urls:
    logger.info(f'Fetching cat url {url}\n')
   
    rss_page =  c.download_page(url)
    if rss_page.startswith('Unable to fetch'):
        logger.info(rss_page) # writes error message with error code
        unable_to_fetch_cat_url += 1
        continue   
# rss_page = c.scrap('class="u-list">(.*?)<div\s*class="page">',rss_page)
    for i in rss_page.split('<li>')[1:]:

        # source_link
        source_link = c.scrap('<a\s*href="(.*?)"', i)
        source_link = source_link.replace("./","")
        if 'http' not in source_link:
            source_link = url + source_link
    
       
        # handle duplicates
        source_link_query = {'source_link':source_link}
        dic = cl_data.find_one(source_link_query,{'source_link': 1}) 
        if dic:
            duplicate_data += 1
            continue

        time.sleep(random.randint(1,3))

        page = c.download_page(source_link)
        if page.startswith('Unable to fetch'):
            logger.info(page) # writes error message with error code
            unable_to_fetch_article_url += 1
            continue    

        # source_headline
        source_headline = c.scrap('<title>(.*?)<', page)
        source_headline = re.sub('【|】|-NDRC_NEW','',source_headline,re.S)

        # skip if headline not found
        if not source_headline:
            logger.info(f'Skipping due to headline {source_link}\n')
            skipped_due_to_headline += 1
            continue

        # Date and time
        pub_date, publish_time = '', ''

        try:
            date_time_str = c.scrap('<meta\s*name="PubDate"\s*content="(.*?)"', page)
            date_time_str = re.sub('[^\d]','',date_time_str,re.S)
            date_time_obj = datetime.strptime(date_time_str, '%Y%m%d%H%M%S')
            ist_date_time = date_time_obj - timedelta(hours = 3,minutes = 30)  # utc time to ist
            ist_date_time = ist_date_time.strftime('%Y-%m-%d %H:%M:%S')
            pub_date = ist_date_time[:10]
            publish_time = ist_date_time[11:]
        except:
            pass

        # skip null date
        if not pub_date:
            logger.info(f'Skipping due to date {source_link}\n')
            skipped_due_to_date += 1
            continue

        # break if date is not today's date
    #     if pub_date != created_on:
    #         break    

        logger.info(f'Fetching {source_link}\n')

        # source_content          
        source_content = c.scrap('class="detail">(.*?)<scrip', page)
        if not source_content:
            source_content = c.scrap('<div\s*class="content">(.*?)<script>', page)
        if not source_content:
            source_content = source_headline
        source_content = c.strip_html(source_content)

        # skip if content not found
        if not source_content:
            logger.info(f'Skipping due to content {source_link}\n')
            skipped_due_to_content += 1
            continue

        # journalist
        journalist = c.scrap('<meta\s*name="Author"\s*content="(.*?)"',page)
        if not journalist: journalist = 'NA'

        # current date and time 
        harvest_time = datetime.now().strftime("%H:%M:%S")

        # temp link
        temp_link = source_link

        # headline and content 
        headline = source_headline
        content = source_content

        # overall_tonality
        overall_tonality = ''

        # word count
        word_count = len((source_headline + ' ' + source_content).split())

        html_content = ''

        # image_urls
        image_urls = []

        # storing the above data in a dictionary
        clientdata ={
                        "client_master" : client_id, 
                        "articleid":client_id,
                        "medium":'Web' ,
                        "searchkeyword":[],
                        "entityname" : [] ,
                        "process_flage":"1",
                        "na_flage":"0",
                        "na_reason":"",
                        "qc_by":"",
                        "qc_on":"",
                        "location":"",
                        "spokeperson":"",
                        "quota":"",
                        "overall_topics":"",
                        "person":"",
                        "overall_entites":"",
                        "overall_tonality": overall_tonality,
                        "overall_wordcount":word_count,
                        "article_subjectivity":"",
                        "article_summary":"",
                        "pub_date":pub_date,
                        "publish_time":publish_time,
                        "harvest_time":harvest_time,
                        "temp_link":temp_link,
                        "publish_source": publish_source,
                        "programme":'null',
                        "feed_class":"News",
                        "publishing_platform":"",
                        "klout_score":"",
                        "journalist":journalist,
                        "headline":headline,
                        "content":content,
                        "source_headline":source_headline,
                        "source_content":source_content,
                        "language":language,
                        "presence":'null',
                        "clip_type":'null',
                        "prog_slot":'null',
                        "op_ed":'0',
                        "location_mention":'',
                        "source_link":source_link,
                        "author_contact":'',
                        "author_emailid":'',
                        "author_url":'',
                        "city":'',
                        "state":'',
                        "country":country,
                        "source":publish_source,
                        "foot_fall":foot_fall,
                        "created_on":created_on,
                        "active":'1',
                        'crawl_flag':2,
                        "images_path":images_path,
                        "html_content":html_content
                    } 

        cl_data.insert_one(clientdata)  # get object id and insert data
        no_of_data += 1


logger.info('Iteration complete\n')   

logger.info(f'Number of data: {no_of_data}\n')
logger.info(f'Duplicate data: {duplicate_data}\n')
logger.info(f'Unable to  fetch cat url: {unable_to_fetch_cat_url }\n')
logger.info(f'Unable to article fetch url: {unable_to_fetch_article_url}\n')
logger.info(f'Skipped due to headline: {skipped_due_to_headline}\n')
logger.info(f'Skipped due to content: {skipped_due_to_content}\n')
logger.info(f'Skipped due to date: {skipped_due_to_date}\n')
logger.info(f'country : {country}\n')
logger.info(f'language : {language}\n')
logger.info(f'Processing finished in {time.time() - start_time} seconds.\n')

https://en.ndrc.gov.cn/news/pressreleases/
https://en.ndrc.gov.cn/news/pressreleases/202107/t20210705_1285405.html
https://en.ndrc.gov.cn/news/pressreleases/202106/t20210623_1283816.html
https://en.ndrc.gov.cn/news/pressreleases/202106/t20210623_1283813.html
https://en.ndrc.gov.cn/news/pressreleases/pressreleasespic/202106/t20210617_1283410.html
https://en.ndrc.gov.cn/news/pressreleases/pressreleasespic/202106/t20210616_1283319.html
https://en.ndrc.gov.cn/news/pressreleases/202106/t20210611_1283097.html
https://en.ndrc.gov.cn/news/pressreleases/202106/t20210611_1283096.html
https://en.ndrc.gov.cn/news/pressreleases/202106/t20210603_1282571.html
https://en.ndrc.gov.cn/news/pressreleases/202105/t20210531_1282255.html
https://en.ndrc.gov.cn/news/pressreleases/202105/t20210531_1282259.html
https://en.ndrc.gov.cn/news/pressreleases/202105/t20210531_1282258.html
https://en.ndrc.gov.cn/news/pressreleases/202105/t20210531_1282261.html
https://en.ndrc.gov.cn/news/pressreleases/202105/t20210531_